# Actor

- Carl Hewitt, 1973
- A mathematical model of concurrent computation

- “一切皆Actor”，高级抽象
- Actor类似于对象，包含状态、行为
- 每个Actor都有一个Mailbox，用于收发消息
- 每个Actor都可以是一个Supervisor，采用特定的监管策略来管理Child Actors的生命周期

![](actor.png)

Actor的行为：
- 发送消息给其它的Actor
- 接收并处理消息，计算，修改自己的状态
- 创建子Actor

![](actors.png)

Actor模型的目标：
- 无缝弥合本地和远程收发消息，天然分布式
- 容错，永不停机，自愈合

Actor模型的特点：
- Actor是主体，Mailbox是透明的
- 只关心消息发送给谁了，不关心消息类型和通道
- 通过模式匹配机制做消息处理

- Actor模型和CSP模型有点相似，又有一些差异
- Actor模型是更高级的抽象，设计目标更大

> 问题来了：在Go的goroutine和channel基础上可以实现Actor模型吗？

**proto.actor**
- http://proto.actor/
- https://github.com/AsynkronIT/protoactor-go

proto.actor的特点：
- API简单易用
- 网络使用gRPC，分布式使用Consul
- 消息格式使用Protobuf，可以跨语言
- 每秒几百万个消息，比Akka快6倍

Actor监管层次：
- 根Actor
- 系统级Actor
- 用户级Actor

![](TopLevelSupervisors.png)

Actor容错策略

![](OneForOne.png)

![](AllForOne.png)

本地收发消息

In [9]:
import (
    "github.com/AsynkronIT/goconsole"
    "github.com/AsynkronIT/protoactor-go/actor"
)

type Hello struct{ Who string }
type HelloActor struct{}

func (state *HelloActor) Receive(context actor.Context) {
    switch msg := context.Message().(type) {
    case Hello:
        fmt.Printf("Hello %v\n", msg.Who)
    }
}

func main() {
    props := actor.FromInstance(&HelloActor{})
    pid := actor.Spawn(props)
    pid.Tell(Hello{Who: "Roger"})
    console.ReadLine()
}

Error: 

Actor生命周期事件消息

In [10]:
type Hello struct{ Who string }
type HelloActor struct{}

func (state *HelloActor) Receive(context actor.Context) {
    switch msg := context.Message().(type) {
    case *actor.Started:
        fmt.Println("Started, initialize actor here")
    case *actor.Stopping:
        fmt.Println("Stopping, actor is about shut down")
    case *actor.Stopped:
        fmt.Println("Stopped, actor and its children are stopped")
    case *actor.Restarting:
        fmt.Println("Restarting, actor is about restart")
    case Hello:
        fmt.Printf("Hello %v\n", msg.Who)
    }
}

func main() {
    props := actor.FromInstance(&HelloActor{})
    actor := actor.Spawn(props)
    actor.Tell(Hello{Who: "Roger"})

    time.Sleep(1 * time.Second)
    actor.Stop()

    console.ReadLine()
}

Error: 